In [4]:
import pandas as pd
import requests

Set up a path variable to access the data

In [2]:
shark_url = "http://api.fish.wa.gov.au/webapi/v1/RawData"
csv_path = "data/sharks/shark_sightings_raw.csv"

Download the data from the API

In [ ]:
response = requests.get(shark_url).json()


Load the data into a Pandas DF

In [5]:
ships_df = pd.read_csv(csv_path)

Save the data into a CSV file

In [ ]:
ships_df.to_csv("data/sharks/shark_sightings_raw.csv")


In [6]:
ships_df

,Unnamed: 0,RawDataId,ObjectId,Distance,DistanceUnit,InteractionValue,InteractionId,TownProximity,LocationValue,SightingNumbervalue,ReportDateTime,SightingDateTime,SightingSizeValue,SightingSpeciesValue,OwnerValue,LocationDetail,LocationX,LocationY
0,0,63739,3073,NaN,NaN,Detected,4,NaN,Meelup,NaN,NaN,2021-07-24T17:48:00,NaN,white,Fisheries Advise,NaN,115.093091,-33.57247
1,1,63738,3073,NaN,NaN,Detected,4,NaN,Meelup,NaN,NaN,2021-07-24T17:43:00,NaN,white,Fisheries Advise,NaN,115.093091,-33.57247
2,2,63737,3073,NaN,NaN,Detected,4,NaN,Meelup,NaN,NaN,2021-07-24T17:38:00,NaN,white,Fisheries Advise,NaN,115.093091,-33.57247
3,3,63736,WA12582TSE,50.0,m offshore,sighted,1,Perth Metro,"Jervoise Bay Beach, Perth Metro",NaN,NaN,2021-07-24T12:15:12,2.5m,unknown sp.,Public report,NaN,115.765000,-32.13930
4,4,63735,WA51495TSE,600.0,m offshore,sighted,1,Coral Bay,"Bateman Bay, near Coral Bay",NaN,NaN,2021-07-24T11:24:29,NaN,whale carcass,Fisheries advise,Lots of sharks around carcass,113.807000,-23.05480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17309,17309,54674,WA00004TSE,10.0,m,Sighted,1,Dunsborough,"Bunker Bay, near Dunsborough",1,NaN,2014-10-14T03:40:00,2m,white,Public report,NaN,115.041100,-33.54306
17310,17310,54675,WA00003TSE,NaN,NaN,Sighted,1,Yallingup,"Yallingup Beach, Yallingup",1,NaN,2014-10-14T02:08:00,NaN,unknown sp.,Public report,NaN,115.028170,-33.63339
17311,17311,54676,WA00002TSE,NaN,NaN,Sighted,1,Dunsborough,"Dunsborough Boat Ramp, Dunsborough",1,2014-10-13T04:00:00,2014-10-13T03:55:00,4.5m,white,Public report,Old Dunsborough,115.105300,-33.60111
17312,17312,46307,1035,NaN,NaN,Detected,4,NaN,Garden Island (north end),NaN,NaN,2014-10-13T02:28:00,NaN,tiger,Fisheries Advise,NaN,115.642830,-32.14411
